In [2]:
import pandas as pd
import logging
from marginal_emissions.utils.helper import search_df, check_encoding
logging.basicConfig(level=logging.INFO, format='[%(levelname)s][%(asctime)s][%(filename)s]_%(message)s')

In [3]:
kws_path = '../data/raw/bundesnetzagentur/kraftwerksliste.csv'
cdc_path = '../data/raw/bundesnetzagentur/ZuUndRueckbau.xlsx' # cdc = commission/decommission
wind_path = '~/Downloads/stundenwerte_FF_00282_19710801_20241231_hist/produkt_ff_stunde_19710801_20241231_00282.txt' # avgeraged by hour
wind_path2 = '~/Downloads/stundenwerte_F_00282_19490101_20241231_hist/produkt_f_stunde_19490101_20241231_00282.txt' # hourly values
ele_em = '../data/raw/electricitymaps/DE_2024_hourly.csv'

In [7]:
kws_encoding = check_encoding(path=kws_path)

# To check exact column names
kws_cols = pd.read_csv(kws_path, sep=';', skiprows=11, header=0, nrows=5, encoding='Windows-1252')

if kws_encoding:
    kws = pd.read_csv(kws_path, sep=';', skiprows=11, header=0, encoding=kws_encoding, usecols=[
        'Anzeige-Name der Stromerzeugungseinheit',
        'Anlagenbetreiber',
        'Anschlussnetzbetreiber',
        'Bundesland der Einheit',
        'Ort der Einheit',
        'Kraftwerksstatus der Einheit',
        'Energieträger',
        'Speichertechnologie',
        'Wärmeauskopplung (KWK)\n(ja/nein)',
        'Nettonennleistung (elektrische Wirkleistung) in MW',
        'Technologie der Stromerzeugung',
        'Spannungsebene'
    ]).rename(columns={
        'Anzeige-Name der Stromerzeugungseinheit': 'Name',
        'Bundesland der Einheit': 'Bundesland',
        'Ort der Einheit': 'Ort',
        'Kraftwerksstatus der Einheit': 'Status',
        'Wärmeauskopplung (KWK)\n(ja/nein)': 'KWK',
        'Nettonennleistung (elektrische Wirkleistung) in MW': 'Net_Wirkleistung',
        'Technologie der Stromerzeugung': 'Technologie'
    })

    kw_carrier = kws[kws['Energieträger'].str.contains('Speicher', case=False, na=False)].drop(columns='Technologie')
    kw_carrier_pump = kw_carrier[kw_carrier['Speichertechnologie'] == 'Pumpspeicher']
    kw_solar = search_df(kws, 'Solar').copy() # use .copy() to avoid modifying
    kw_solar.Net_Wirkleistung = kw_solar.Net_Wirkleistung.str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float) # makes float from string
    kw_wind = kws[kws['Energieträger'].str.contains('wind', case=False, na=False)].copy()
    kw_wind.Net_Wirkleistung = kw_wind.Net_Wirkleistung.str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float) # makes float from string
    kw_kwk = kws[kws['KWK'] == 'Ja']

    print("# Kraftwerke: ", len(kws))
    print("# Speicher: ", len(kw_carrier_pump))
    print("# Must-Run: ", len(kw_kwk))

    # Einzeiler für eine ganze Spalte
    # df['column_name'] = df['column_name'].str.replace('.', '', regex=False).str.replace(',', '.').astype(float)

[INFO][2026-01-08 13:49:20][helper.py] Checking encoding of file "../data/raw/bundesnetzagentur/kraftwerksliste.csv"...
[INFO][2026-01-08 13:49:20][helper.py] File is encoded as "Windows-1252"
[INFO][2026-01-08 13:49:20][helper.py] Found 22 rows matching 'Solar' pattern.
# Kraftwerke:  2220
# Speicher:  114
# Must-Run:  1158


In [9]:
kws['Anschlussnetzbetreiber'].unique()


array(['Regionetz GmbH (SNB911641710114)',
       'Netze BW GmbH (SNB948311994307)',
       '50Hertz Transmission GmbH (SNB982046657236)',
       'Amprion GmbH (SNB976890256486)',
       'Überlandwerk Leinetal GmbH (SNB910956210043)',
       'TransnetBW GmbH (SNB984944755380)',
       'LEW Verteilnetz GmbH (SNB911705062982)',
       'TEN Thüringer Energienetze GmbH & Co. KG (SNB970821959712)',
       'E.DIS Netz GmbH (SNB941690671609)',
       'Pfalzwerke Netz AG (SNB961471621746)',
       'Infrastrukturbetrieb der Stadt Arneburg (SNB971962135690)',
       'Westnetz GmbH (SNB921897286493)',
       'Aschaffenburger Versorgungs-GmbH (SNB911081401368)',
       'BIGGE ENERGIE GmbH & Co. KG (SNB990174285078)',
       'swa Netze GmbH (SNB958077102830)',
       'Bayernwerk Netz GmbH (SNB940352624434)',
       'Mitteldeutsche Netzgesellschaft Strom mbH (SNB916269213931)',
       'Avacon Netz GmbH (SNB990362338043)',
       'DB Energie GmbH (Bahnstromnetz 16,7 Hz) (SNB942182027607)',
       'GS

In [34]:
kws_cols.columns

Index(['MaStR-Nr. der Stromerzeugungseinheit', 'Anlagenbetreiber',
       'Anzeige-Name der Stromerzeugungseinheit', 'PLZ der Einheit',
       'Ort der Einheit', 'Straße der Einheit', 'Hausnummer der Einheit',
       'Bundesland der Einheit',
       'Datum der erstmaligen Inbetriebnahme der Einheit',
       'Jahr der Inbetriebnahme der Einheit', 'Kraftwerksstatus der Einheit',
       'Energieträger', 'Hauptbrennstoff', 'Speichertechnologie',
       'Auswertung Energieträger', 'Wärmeauskopplung (KWK)\n(ja/nein)',
       'Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?',
       'Bruttoleistung in MW',
       'Nettonennleistung (elektrische Wirkleistung) in MW',
       'Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?: ja \nNettonennleistung der Einspeisung in ein deutsches Netz:',
       'Technologie der Stromerzeugung',
       'Volleinspeisung oder Teileinspeisung?', 'Anschlussnetzbetreiber',
       'Spannungsebene'],
      dtype='object')

In [193]:
net_solar = kw_solar.Net_Wirkleistung.sum()
net_wind = kw_wind.Net_Wirkleistung.sum()

print(f"Windeinspeisung: {net_wind} MWh; Solareinspeisung: {net_solar} MWh")

Windeinspeisung: 72804.1 MWh; Solareinspeisung: 89482.94 MWh


In [125]:
# Wetterdaten
import csv
wind_bam = pd.read_csv(wind_path, delimiter=';').rename(columns={
        'STATIONS_ID': 'id',
        'MESS_DATUM': 'timestamp',
        'QN_3': 'quality_lvl',
        '   F': 'speed',
        '   D': 'direction'
    }).drop(columns=['eor'])
wind_bam.timestamp = pd.to_datetime(wind_bam.timestamp, format='%Y%m%d%H')
wind_bam = wind_bam[wind_bam.timestamp.dt.year >= 2000]
wind_bam.set_index('timestamp')

wind_bam2 = pd.read_csv(wind_path2, delimiter=';').rename(columns={
    'STATIONS_ID': 'id',
    'MESS_DATUM': 'timestamp',
    'QN_8': 'quality_lvl',
    '  FF': 'speed',
    '  DD': 'direction'
}).drop(columns=['eor'])
wind_bam2.timestamp = pd.to_datetime(wind_bam2.timestamp, format='%Y%m%d%H')
wind_bam2 = wind_bam2[wind_bam2.timestamp.dt.year >= 2000]
wind_bam2.set_index('timestamp')

,id,quality_lvl,speed,direction
timestamp,,,,
2000-01-01 00:00:00,282,1,2.0,130
2000-01-01 01:00:00,282,1,2.0,150
2000-01-01 02:00:00,282,1,3.0,140
2000-01-01 03:00:00,282,1,2.0,160
2000-01-01 04:00:00,282,1,3.0,160
...,...,...,...,...
2024-12-31 19:00:00,282,3,1.5,150
2024-12-31 20:00:00,282,3,4.1,150
2024-12-31 21:00:00,282,3,2.1,160


In [126]:
wind_bam[wind_bam.speed == wind_bam.speed.max()]

,id,timestamp,quality_lvl,speed,direction
381752,282,2015-03-31 11:00:00,10,11.7,270


In [135]:
wind_bam2[(wind_bam2.speed == wind_bam2.speed.max()) & (wind_bam2.quality_lvl > 0)]

,id,timestamp,quality_lvl,speed,direction
193647,282,2000-01-20 17:00:00,1,21.0,0


In [142]:
wind_bam2[(wind_bam2.quality_lvl > 2) & (wind_bam2.speed > 10)]

,id,timestamp,quality_lvl,speed,direction
242034,282,2005-07-29 20:00:00,3,10.8,210
250742,282,2006-07-27 16:00:00,3,10.3,160
254945,282,2007-01-18 19:00:00,3,11.5,240
254950,282,2007-01-19 00:00:00,3,10.8,290
254951,282,2007-01-19 01:00:00,3,11.3,290
264714,282,2008-03-01 15:00:00,3,11.4,280
264734,282,2008-03-02 11:00:00,3,10.4,270
273015,282,2009-02-10 12:00:00,3,11.4,220
282210,282,2010-02-28 15:00:00,3,10.7,250
282211,282,2010-02-28 16:00:00,3,11.1,230


In [110]:
max_speed

,id,timestamp,quality_lvl,speed,direction
273121,282,2002-10-27 22:00:00,10,10.4,260
310086,282,2007-01-18 16:00:00,10,10.3,250
310090,282,2007-01-18 20:00:00,10,10.3,250
310093,282,2007-01-18 23:00:00,10,10.3,290
319832,282,2008-03-01 15:00:00,10,10.2,280
337330,282,2010-02-28 17:00:00,10,10.3,240
337952,282,2010-03-26 15:00:00,10,10.5,240
353424,282,2012-01-05 11:00:00,10,10.4,240
381749,282,2015-03-31 08:00:00,10,10.4,250
381750,282,2015-03-31 09:00:00,10,10.8,260


In [3]:
# Emissions data electricitymaps
df_emissions = pd.read_csv(ele_em)

In [4]:
df_emissions.head()

,Datetime (UTC),Country,Zone name,Zone id,Carbon intensity gCO₂eq/kWh (direct),Carbon intensity gCO₂eq/kWh (Life cycle),Carbon-free energy percentage (CFE%),Renewable energy percentage (RE%),Data source,Data estimated,Data estimation method
0,2024-01-01 00:00:00,Germany,Germany,DE,134.25,178.76,82.96,81.23,entsoe.eu,False,NaN
1,2024-01-01 01:00:00,Germany,Germany,DE,135.33,180.12,82.81,81.47,entsoe.eu,False,NaN
2,2024-01-01 02:00:00,Germany,Germany,DE,139.14,184.74,82.32,81.29,entsoe.eu,False,NaN
3,2024-01-01 03:00:00,Germany,Germany,DE,140.23,186.02,82.22,81.48,entsoe.eu,False,NaN
4,2024-01-01 04:00:00,Germany,Germany,DE,140.56,186.58,82.16,81.62,entsoe.eu,False,NaN


In [12]:
df_emissions['Carbon intensity gCO₂eq/kWh (direct)'].min()

66.85